# BiteMe | Train

This notebook includes the most important part of the project - the modelling. The notebook tests methodologies for training, and in it the chosen algorithm is decided. Validation also occurs before final testing, which is conducted in the test notebook. This stage is highly iterative, so all model artefacts, logs and configurations are recorded and saved to disk automatically. This initial setup of what will eventually become MLOps for the final product will be really useful, and helps keep track of what is successful and what isn't.

Models to try:

~~[SE-ResNet50](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet101](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SE-ResNet152](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[SENet154](https://github.com/Cadene/pretrained-models.pytorch#senet)~~
~~[ResNet34](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet50](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet101](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
~~[ResNet152](https://github.com/Cadene/pretrained-models.pytorch#torchvision)~~
 - [FBResNet152](https://github.com/Cadene/pretrained-models.pytorch#facebook-resnet)
 - [VGG16](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG16_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [VGG19_BN](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [PolyNet](https://github.com/Cadene/pretrained-models.pytorch#polynet)
 - [InceptionResNetV2](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [InceptionV4](https://github.com/Cadene/pretrained-models.pytorch#inception)
 - [Xception](https://github.com/Cadene/pretrained-models.pytorch#xception)
 - [ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [ResNeXt101_64x4d](https://github.com/Cadene/pretrained-models.pytorch#resnext)
 - [SE-ResNeXt50_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [SE-ResNeXt101_32x4d](https://github.com/Cadene/pretrained-models.pytorch#senet)
 - [DenseNet121](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet161](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet169](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DenseNet201](https://github.com/Cadene/pretrained-models.pytorch#torchvision)
 - [DualPathNet68](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet92](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet98](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet107](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [DualPathNet131](https://github.com/Cadene/pretrained-models.pytorch#dualpathnetworks)
 - [NASNet-A-Large](https://github.com/Cadene/pretrained-models.pytorch#nasnet)
 - [PNASNet-5-Large](https://github.com/Cadene/pretrained-models.pytorch#pnasnet)


 - efficientnet_b0
 - efficientnet_b1
 - efficientnet_b2
 - efficientnet_b3
 - efficientnet_b4
 - efficientnet_b5

Initial model work is done by using simple, typical image recognition models (CNN architectures) to see how effective these models can be for the problem. Although I don't expect them to be particularly successful, it's important to establish baselines and take a holistic approach to modelling when it's possible.

In [1]:
# Basic imports
import pandas as pd
import numpy as np
import os
import sys
from argparse import ArgumentParser
import datetime
from time import time
import gc
from tqdm import tqdm

# Data visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn

# Image processing
import cv2
import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

# Model evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score

import torch
import pretrainedmodels
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# Local imports
sys.path.append("..")
from utils.dataset import generate_transforms, generate_dataloaders
from models.models import *
from utils.loss_function import CrossEntropyLossOneHot
from utils.lrs_scheduler import WarmRestart, warm_restart
from utils.utils import read_images, augs, get_augs, seed_reproducer, init_logger
from utils.constants import *

plt.rcParams["figure.figsize"] = (14, 8)

In [2]:
# Define directories
base_dir_path = "../"

data_dir_path = os.path.join(base_dir_path, "data")
data_preprocessed_dir_path = os.path.join(data_dir_path, "preprocessed")
data_preprocessed_train_dir_path = os.path.join(data_dir_path, "preprocessed/train")

data_dir = os.listdir(data_dir_path)
data_preprocessed_dir = os.listdir(data_preprocessed_dir_path)
data_preprocessed_train_dir = os.listdir(data_preprocessed_train_dir_path)

metadata_preprocessed_path = os.path.join(data_preprocessed_dir_path, "metadata.csv")
metadata = pd.read_csv(metadata_preprocessed_path)
# Subset to train only
metadata = metadata.loc[metadata.split == "train"]

metadata.head()

,img_name,img_path,label,split
0,7059b14d2aa03ed6c4de11afa32591995181d31c.jpg,../data/cleaned/none/7059b14d2aa03ed6c4de11afa...,none,train
1,ea1b100b581fcdb7ddfae52cc62347a99e304ba4.jpg,../data/cleaned/none/ea1b100b581fcdb7ddfae52cc...,none,train
2,6eac051b9c45ff6821ec8675216f371711b7cea9.jpg,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,train
3,fc72767f8520df9b2b83941077dc0ee013eb9399.jpg,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,train
4,49850884a00703afe5aab78c3ce074d2d4acae30.jpg,../data/cleaned/none/49850884a00703afe5aab78c3...,none,train


In [3]:
# Read in train images
X_train = read_images(
    data_dir_path=data_preprocessed_train_dir_path, 
    rows=ROWS, 
    cols=COLS, 
    channels=CHANNELS, 
    write_images=False, 
    output_data_dir_path=None,
    verbose=VERBOSE
)

# Get labels
y_train = np.array(pd.get_dummies(metadata["label"]))

Reading images from: ../data/preprocessed/train
Rows set to 1024
Columns set to 1024
Channels set to 3
Writing images is set to: False
Reading images...


100%|███████████████████████████████████████████| 46/46 [00:08<00:00,  5.22it/s]


Image reading complete.
Image array shape: (299, 1024, 1024, 3)


## Set Parameters

In [4]:
# Choose augmentations to use in preprocessing
# For full list see helpers.py
#augs_to_select = [
#    "Resize",
#    "HorizontalFlip", 
#    "VerticalFlip",
#    "Normalize"
#]
## Subset augs based on those selected
#AUGS = dict((aug_name, augs[aug_name]) for aug_name in augs_to_select)


def init_hparams():
    """
    Initialise hyperparameters for modelling.
    
    Returns
    ---------
    hparams : argparse.Namespace
        Parsed hyperparameters
    """
    parser = ArgumentParser(add_help=False)
    parser.add_argument("-backbone", "--backbone", type=str, default=MODEL_NAME)
    parser.add_argument("-device_name", type=str, default=DEVICE_NAME)
    parser.add_argument("--gpus", default=[0])
    parser.add_argument("--n_workers", type=int, default=N_WORKERS)
    parser.add_argument("--image_size", nargs="+", default=[ROWS, COLS])
    parser.add_argument("--seed", type=int, default=SEED)
    parser.add_argument("--min_epochs", type=int, default=MIN_EPOCHS)
    parser.add_argument("--max_epochs", type=int, default=MAX_EPOCHS)
    parser.add_argument("--patience", type=str, default=PATIENCE)    
    parser.add_argument("-tbs", "--train_batch_size", type=int, default=TRAIN_BATCH_SIZE)
    parser.add_argument("-vbs", "--val_batch_size", type=int, default=VAL_BATCH_SIZE)
    parser.add_argument("--n_splits", type=int, default=N_SPLITS)
    parser.add_argument("--test_size", type=float, default=TEST_SIZE)
    parser.add_argument("--precision", type=int, default=PRECISION)
    parser.add_argument("--gradient_clip_val", type=float, default=GRADIENT_CLIP_VAL)
    parser.add_argument("--verbose", type=str, default=VERBOSE)
    parser.add_argument("--log_dir", type=str, default=LOG_DIR)
    parser.add_argument("--log_name", type=str, default=LOG_NAME)
    
    
    try:
        hparams, unknown = parser.parse_known_args()
    except:
        hparams, unknown = parser.parse_args([])

    if len(hparams.gpus) == 1:
        hparams.gpus = [int(hparams.gpus[0])]
    else:
        hparams.gpus = [int(gpu) for gpu in hparams.gpus]

    hparams.image_size = [int(size) for size in hparams.image_size]
    
    return hparams

### Create Model

In [5]:
class CoolSystem(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams

        seed_reproducer(self.hparams.seed)

        self.model = fbresnet152()
        self.criterion = CrossEntropyLossOneHot()
        self.logger_kun = init_logger(
            hparams.log_name, 
            hparams.log_dir
        )

    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0
        )
        self.scheduler = WarmRestart(self.optimizer, T_max=10, T_mult=3, eta_min=1e-5)
        return [self.optimizer], [self.scheduler]

    def training_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch

        scores = self(images)
        loss = self.criterion(scores, labels)

        data_load_time = torch.sum(data_load_time)

        return {
            "loss": loss,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def training_epoch_end(self, outputs):
        # outputs is the return of training_step
        train_loss_mean = torch.stack([output["loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        self.current_epoch += 1
        if self.current_epoch < (self.trainer.max_epochs - 4):
            self.scheduler = warm_restart(self.scheduler, T_mult=2)

        return {"train_loss": train_loss_mean}

    def validation_step(self, batch, batch_idx):
        step_start_time = time()
        images, labels, data_load_time = batch
        data_load_time = torch.sum(data_load_time)
        scores = self(images)
        loss = self.criterion(scores, labels)

        # must return key -> val_loss
        return {
            "val_loss": loss,
            "scores": scores,
            "labels": labels,
            "data_load_time": data_load_time,
            "batch_run_time": torch.Tensor([time() - step_start_time + data_load_time]).to(
                data_load_time.device
            ),
        }

    def validation_epoch_end(self, outputs):
        # compute loss
        val_loss_mean = torch.stack([output["val_loss"] for output in outputs]).mean()
        self.data_load_times = torch.stack([output["data_load_time"] for output in outputs]).sum()
        self.batch_run_times = torch.stack([output["batch_run_time"] for output in outputs]).sum()

        # compute roc_auc
        scores_all = torch.cat([output["scores"] for output in outputs]).cpu()
        labels_all = torch.round(torch.cat([output["labels"] for output in outputs]).cpu())

        val_roc_auc = torch.tensor(roc_auc_score(labels_all, scores_all))

        # terminal logs
        self.logger_kun.info(
            f"{self.hparams.fold_i}-{self.current_epoch} | "
            f"lr : {self.scheduler.get_lr()[0]:.6f} | "
            f"val_loss : {val_loss_mean:.4f} | "
            f"val_roc_auc : {val_roc_auc:.4f} | "
            f"data_load_times : {self.data_load_times:.2f} | "
            f"batch_run_times : {self.batch_run_times:.2f}"
        )

        return {"val_loss": val_loss_mean, "val_roc_auc": val_roc_auc}

## Cross Validation

In [6]:
# Initialise hyperparameters
hparams = init_hparams()
torch.cuda.empty_cache()

log_notes = "decreased gradient_clip_val to 0.5"

# Initialise logger
logger = init_logger(hparams.log_name, hparams.log_dir)

# Log parameters
logger.info(f"backbone: {hparams.backbone}")
logger.info(f"device_name: {hparams.device_name}")
logger.info(f"gpus: {hparams.gpus}")
logger.info(f"n_workers: {hparams.n_workers}")
logger.info(f"image_size: {hparams.image_size}")
logger.info(f"seed: {hparams.seed}")
logger.info(f"min_epochs: {hparams.min_epochs}")
logger.info(f"max_epochs: {hparams.max_epochs}")
logger.info(f"patience: {hparams.patience}")
logger.info(f"train_batch_size: {hparams.train_batch_size}")
logger.info(f"val_batch_size: {hparams.val_batch_size}")
logger.info(f"n_splits: {hparams.n_splits}")
logger.info(f"test_size: {hparams.test_size}")
logger.info(f"precision: {hparams.precision}")
logger.info(f"gradient_clip_val: {hparams.gradient_clip_val}")
logger.info(f"log_dir: {hparams.log_dir}")
logger.info(f"log_name: {hparams.log_name}")

# Log any notes if they exist
if "log_notes" in locals():
    logger.info(f"Notes: {log_notes}")


# Create transform pipeline
transforms = generate_transforms(hparams.image_size)

# List for validation scores 
val_loss_scores = []

# Initialise cross validation
folds = StratifiedKFold(n_splits=hparams.n_splits, shuffle=True, random_state=hparams.seed)

# Start cross validation
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    hparams.fold_i = fold_i
    # Split train images and validation sets
    train_data = metadata.iloc[train_index][["img_path", "label"]].reset_index(drop=True)
    train_data = pd.get_dummies(train_data, columns=["label"], prefix="", prefix_sep="")

    val_data = metadata.iloc[val_index][["img_path", "label"]].reset_index(drop=True)
    val_data = pd.get_dummies(val_data, columns=["label"], prefix="", prefix_sep="")
    
    logger.info(f"Fold {fold_i} num train records: {train_data.shape[0]}")
    logger.info(f"Fold {fold_i} num val records: {val_data.shape[0]}")
    
    train_dataloader, val_dataloader = generate_dataloaders(hparams, train_data, val_data, transforms)
    
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        save_top_k=2,
        mode="min",
        filepath=os.path.join(
            hparams.log_dir, 
            hparams.log_name, 
            f"fold={fold_i}" + "-{epoch}-{val_loss:.4f}-{val_roc_auc:.4f}"
        )
    )
    
    early_stop_callback = EarlyStopping(
        monitor="val_loss", 
        patience=hparams.patience, 
        mode="min", 
        verbose=hparams.verbose
    )
    
    # Instance Model, Trainer and train model
    model = CoolSystem(hparams)
    trainer = pl.Trainer(
        gpus=hparams.gpus,
        min_epochs=hparams.min_epochs,
        max_epochs=hparams.max_epochs,
        early_stop_callback=early_stop_callback,
        checkpoint_callback=checkpoint_callback,
        progress_bar_refresh_rate=0,
        precision=hparams.precision,
        num_sanity_val_steps=0,
        profiler=False,
        weights_summary=None,
        gradient_clip_val=hparams.gradient_clip_val,
        default_root_dir=os.path.join(hparams.log_dir, hparams.log_name)
    )
    
    # Fit model
    trainer.fit(model, train_dataloader, val_dataloader)
            
    # Save val scores
    val_loss_scores.append(checkpoint_callback.best)
    
    # Cleanup
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
val_loss_scores = [i.item() for i in val_loss_scores]

# Add val scores to csv with all scores
if os.path.isfile("../logs/scores.csv") == False:
    pd.DataFrame(columns=["name", "scores", "mean_score"]).to_csv("../logs/scores.csv", index=False)
    
# Append to current scores csv
all_scores_df = pd.concat([
    pd.read_csv("../logs/scores.csv"),
    pd.DataFrame.from_dict(
        {
            "name": [hparams.log_name],
            "scores": [val_loss_scores],
            "mean_score": [np.mean(val_loss_scores)]
        }
    )],
    ignore_index=True
)
# Write all scores df to csv
all_scores_df.to_csv("../logs/scores.csv", index=False)

logger.info(f"Best scores: {val_loss_scores}")
logger.info("Training complete.")

[2022-11-23 20:26:37] 4182638920.py[  11] : INFO  backbone: fbresnet152
[2022-11-23 20:26:37] 4182638920.py[  12] : INFO  device_name: NVIDIA GeForce RTX 3090
[2022-11-23 20:26:37] 4182638920.py[  13] : INFO  gpus: [0]
[2022-11-23 20:26:37] 4182638920.py[  14] : INFO  n_workers: 128
[2022-11-23 20:26:37] 4182638920.py[  15] : INFO  image_size: [1024, 1024]
[2022-11-23 20:26:37] 4182638920.py[  16] : INFO  seed: 14
[2022-11-23 20:26:37] 4182638920.py[  17] : INFO  min_epochs: 30
[2022-11-23 20:26:37] 4182638920.py[  18] : INFO  max_epochs: 50
[2022-11-23 20:26:37] 4182638920.py[  19] : INFO  patience: 11
[2022-11-23 20:26:37] 4182638920.py[  20] : INFO  train_batch_size: 4
[2022-11-23 20:26:37] 4182638920.py[  21] : INFO  val_batch_size: 4
[2022-11-23 20:26:37] 4182638920.py[  22] : INFO  n_splits: 3
[2022-11-23 20:26:37] 4182638920.py[  23] : INFO  test_size: 0.1
[2022-11-23 20:26:37] 4182638920.py[  24] : INFO  precision: 16
[2022-11-23 20:26:37] 4182638920.py[  25] : INFO  gradient_c

[2022-11-23 20:57:55] 3799136884.py[  85] : INFO  0-41 | lr : 0.000100 | val_loss : 1.5059 | val_roc_auc : 0.8258 | data_load_times : 45.29 | batch_run_times : 46.31
[2022-11-23 20:58:40] 3799136884.py[  85] : INFO  0-42 | lr : 0.000100 | val_loss : 1.5270 | val_roc_auc : 0.8221 | data_load_times : 44.61 | batch_run_times : 45.63
[2022-11-23 20:59:24] 3799136884.py[  85] : INFO  0-43 | lr : 0.000100 | val_loss : 1.4758 | val_roc_auc : 0.8131 | data_load_times : 44.97 | batch_run_times : 45.99
[2022-11-23 21:00:08] 3799136884.py[  85] : INFO  0-44 | lr : 0.000100 | val_loss : 1.5434 | val_roc_auc : 0.8004 | data_load_times : 44.31 | batch_run_times : 45.34
[2022-11-23 21:00:52] 3799136884.py[  85] : INFO  0-45 | lr : 0.000099 | val_loss : 1.5737 | val_roc_auc : 0.7936 | data_load_times : 45.37 | batch_run_times : 46.39
[2022-11-23 21:01:36] 3799136884.py[  85] : INFO  0-46 | lr : 0.000099 | val_loss : 1.6066 | val_roc_auc : 0.7523 | data_load_times : 46.73 | batch_run_times : 47.76
[202

[2022-11-23 21:35:34] 3799136884.py[  85] : INFO  1-41 | lr : 0.000100 | val_loss : 1.5091 | val_roc_auc : 0.8175 | data_load_times : 47.26 | batch_run_times : 48.31
[2022-11-23 21:36:21] 3799136884.py[  85] : INFO  1-42 | lr : 0.000100 | val_loss : 1.5073 | val_roc_auc : 0.8332 | data_load_times : 46.43 | batch_run_times : 47.46
[2022-11-23 21:37:08] 3799136884.py[  85] : INFO  1-43 | lr : 0.000100 | val_loss : 1.5268 | val_roc_auc : 0.8435 | data_load_times : 45.29 | batch_run_times : 46.41
[2022-11-23 21:37:55] 3799136884.py[  85] : INFO  1-44 | lr : 0.000100 | val_loss : 1.5416 | val_roc_auc : 0.8250 | data_load_times : 46.24 | batch_run_times : 47.27
[2022-11-23 21:38:42] 3799136884.py[  85] : INFO  1-45 | lr : 0.000099 | val_loss : 1.4706 | val_roc_auc : 0.8397 | data_load_times : 46.12 | batch_run_times : 47.34
[2022-11-23 21:39:29] 3799136884.py[  85] : INFO  1-46 | lr : 0.000099 | val_loss : 1.4706 | val_roc_auc : 0.8404 | data_load_times : 45.66 | batch_run_times : 46.71
[202

[2022-11-23 22:13:49] 3799136884.py[  85] : INFO  2-39 | lr : 0.000010 | val_loss : 1.5064 | val_roc_auc : 0.8420 | data_load_times : 44.51 | batch_run_times : 45.55
[2022-11-23 22:14:38] 3799136884.py[  85] : INFO  2-40 | lr : 0.000100 | val_loss : 1.6312 | val_roc_auc : 0.7956 | data_load_times : 43.84 | batch_run_times : 45.02
[2022-11-23 22:15:25] 3799136884.py[  85] : INFO  2-41 | lr : 0.000100 | val_loss : 1.5881 | val_roc_auc : 0.8015 | data_load_times : 46.26 | batch_run_times : 47.50
[2022-11-23 22:16:12] 3799136884.py[  85] : INFO  2-42 | lr : 0.000100 | val_loss : 1.5916 | val_roc_auc : 0.7899 | data_load_times : 43.32 | batch_run_times : 44.33
[2022-11-23 22:16:59] 3799136884.py[  85] : INFO  2-43 | lr : 0.000100 | val_loss : 1.5220 | val_roc_auc : 0.8134 | data_load_times : 46.29 | batch_run_times : 47.31
[2022-11-23 22:17:46] 3799136884.py[  85] : INFO  2-44 | lr : 0.000100 | val_loss : 1.6204 | val_roc_auc : 0.8149 | data_load_times : 44.57 | batch_run_times : 45.66
[202

## Validation Inference

In [7]:
# Get model run path and define chosen fold
log_dir = "../logs/logs"
#model_run = "2022_11_08_14:57:52"
model_run = hparams.log_name
model_run_path = os.path.join(log_dir, model_run)
#best_fold = 1
best_fold = val_loss_scores.index(min(val_loss_scores))

# Get best model for chosen fold
model_run_dir = os.listdir(model_run_path)
model_folds = [i for i in model_run_dir if i.startswith(f"fold={best_fold}")]
model_folds_scores = [float(i.split("val_loss=")[1].split("-")[0]) for i in model_folds]
model_name = model_folds[model_folds_scores.index(min(model_folds_scores))]
model_path = os.path.join(model_run_path, model_name)

# Load fold's model
model = CoolSystem(hparams)
model.load_state_dict(
    torch.load(model_path)["state_dict"]
)
model.eval()

# Retrieve validation indices for chosen fold
for fold_i, (train_index, val_index) in enumerate(folds.split(metadata[["img_path"]], metadata[["label"]])):
    if fold_i == best_fold:
        break

# Select fold validation images
X_val = torch.from_numpy(X_train[val_index]).permute(0, 3, 1, 2).float()

# Create predictions looped by batch
counter = 0
val_i_batch = []
val_idx_batch = []
scores_df = pd.DataFrame()

for i, idx in tqdm(enumerate(val_index)):
    counter += 1
    val_i_batch.append(i) # arrays don't preserve index so need ordered index values
    val_idx_batch.append(idx) # for preserved index
    
    # Run inference for val_batch_size
    if counter == hparams.val_batch_size:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()
        # Reset counter and batch
        counter = 0
        val_i_batch = []
        val_idx_batch = []
        
    # Run inference for remaining batch
    elif idx == val_index[-1]:
        preds = model(X_val[val_i_batch])
        
        # Create activation output
        log_softmax = torch.nn.LogSoftmax(dim=-1)

        # Convert raw output to probabilities
        preds = np.exp(log_softmax(preds).detach().numpy())

        # Create df with img paths and predicted label probs
        scores_df_batch = pd.DataFrame(preds, columns=val_data.columns[1:])
        scores_df_batch = pd.merge(
            metadata.iloc[val_idx_batch, 1:3].reset_index(drop=True),
            scores_df_batch, 
            left_index=True,
            right_index=True
        )
        scores_df = pd.concat([scores_df, scores_df_batch], ignore_index=True, axis=0)

        # Cleanup
        gc.collect()
        torch.cuda.empty_cache()

        
# Write predictions to log
scores_df.to_csv(
    os.path.join(model_run_path, f"{model_run}_preds_fold_{best_fold}.csv"),
    index=False
)

100it [05:24,  3.25s/it]


In [8]:
scores_df

,img_path,label,ant,bedbug,bee,horsefly,mite,mosquito,none,tick
0,../data/cleaned/none/6eac051b9c45ff6821ec86752...,none,0.199594,0.179132,0.096645,0.177215,0.122495,0.069613,0.087695,0.067611
1,../data/cleaned/none/fc72767f8520df9b2b8394107...,none,0.199738,0.179867,0.092610,0.194288,0.117436,0.066435,0.084611,0.065015
2,../data/cleaned/none/49850884a00703afe5aab78c3...,none,0.211779,0.174707,0.095123,0.180009,0.121242,0.067141,0.084959,0.065040
3,../data/cleaned/none/8dab219e5fa4d9a05d42b6f3a...,none,0.200346,0.179968,0.094964,0.182751,0.119469,0.069074,0.086861,0.066567
4,../data/cleaned/none/d9c0f49c789a29fb55370319e...,none,0.192123,0.181109,0.097990,0.174168,0.123287,0.071402,0.089708,0.070213
...,...,...,...,...,...,...,...,...,...,...
95,../data/cleaned/ant/b7d95c4a5fe54f5bbfb3442ceb...,ant,0.200831,0.177721,0.097698,0.173661,0.124476,0.069470,0.087910,0.068233
96,../data/cleaned/ant/8fadf11644c8c4877b64bb61cb...,ant,0.199049,0.178787,0.097005,0.176380,0.122680,0.070051,0.088188,0.067861
97,../data/cleaned/ant/ab1b890b774b4cc6b71459a595...,ant,0.189615,0.181847,0.098680,0.171040,0.124264,0.072378,0.090936,0.071240
98,../data/cleaned/ant/d170bac72faf6447bd8d192744...,ant,0.193033,0.181599,0.097713,0.175282,0.123042,0.071082,0.088928,0.069321


## Validation Analysis

In [9]:
print(f"{len(scores_df['img_path'].unique())} unique image paths.")

100 unique image paths.


In [10]:
print("Validation label counts:")
print(scores_df["label"].value_counts())

Validation label counts:
bedbug      19
tick        18
mosquito    16
ant         16
none         9
horsefly     8
mite         7
bee          7
Name: label, dtype: int64


In [11]:
print("Validation prediction counts:")
print(
    pd.melt(
        scores_df,
        id_vars=["img_path", "label"],
        value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
        var_name="pred_label",
        value_name="pred_prob"
    ).sort_values(["img_path", "pred_prob"], ascending=False) \
    .groupby(["img_path", "label"]).first()["pred_label"] \
    .value_counts()
)

Validation prediction counts:
ant    100
Name: pred_label, dtype: int64


In [12]:
# Probability stats by label
pd.concat(
    [
        pd.DataFrame(scores_df.iloc[:, 2:].mean(), columns=["mean"]),
        pd.DataFrame(scores_df.iloc[:, 2:].std(), columns=["std"]),
        pd.DataFrame(scores_df.iloc[:, 2:].min(), columns=["min"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.25)),
        pd.DataFrame(scores_df.iloc[:, 2:].median(), columns=["median"]),
        pd.DataFrame(scores_df.iloc[:, 2:].quantile(0.75)),
        pd.DataFrame(scores_df.iloc[:, 2:].max(), columns=["max"])
    ], 
    axis=1
)

,mean,std,min,0.25,median,0.75,max
ant,0.199900,0.006673,0.188495,0.194659,0.199329,0.203630,0.221213
bedbug,0.179407,0.002472,0.171533,0.177821,0.179512,0.181023,0.185112
bee,0.096582,0.001493,0.092610,0.095749,0.097073,0.097668,0.098680
horsefly,0.176606,0.004302,0.170636,0.173639,0.175626,0.178308,0.194288
mite,0.122535,0.001997,0.116238,0.121634,0.123049,0.124008,0.124980
mosquito,0.069644,0.001532,0.065665,0.068604,0.069802,0.070717,0.072880
none,0.087705,0.001705,0.083456,0.086364,0.087930,0.088979,0.091231
tick,0.067621,0.001882,0.063009,0.066332,0.067964,0.068993,0.071240


In [13]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).pivot_table(
    index=["label"],
    columns=["pred_label"],
    aggfunc="mean"
)

pred_prob                                                    \
pred_label       ant    bedbug       bee  horsefly      mite  mosquito   
label                                                                    
ant         0.202127  0.178731  0.096133  0.177521  0.122194  0.069180   
bedbug      0.200962  0.179077  0.096468  0.177048  0.122363  0.069387   
bee         0.195422  0.181255  0.096886  0.176624  0.122258  0.070567   
horsefly    0.199415  0.180163  0.096397  0.176407  0.122492  0.069691   
mite        0.202051  0.179027  0.096289  0.176395  0.122491  0.069328   
mosquito    0.197185  0.179948  0.097340  0.174679  0.123302  0.070385   
none        0.200798  0.178829  0.095862  0.179979  0.121492  0.068884   
tick        0.199886  0.179255  0.096865  0.175516  0.123005  0.069791   

                                
pred_label      none      tick  
label                           
ant         0.087174  0.066941  
bedbug      0.087324  0.067370  
bee         0.088531  0.068458  
horsefly    0.087932  0.067502  
mite        0.087424  0.066995  
mosquito    0.088579  0.068581  
none        0.086986  0.067169  
tick        0.087848  0.067834

In [14]:
pd.melt(
    scores_df,
    id_vars=["img_path", "label"],
    value_vars=["ant", "bedbug", "bee", "horsefly", "mite", "mosquito" ,"none", "tick"],
    var_name="pred_label",
    value_name="pred_prob"
).sort_values(["img_path", "pred_prob"], ascending=False).groupby(["img_path", "label"]).first()

,,pred_label,pred_prob
img_path,label,,
../data/cleaned/ant/24aca08f4aeea83c077a9721c85b5a6ff8ef7d3d.jpg,ant,ant,0.201319
../data/cleaned/ant/26b8fb30b3ddd29f8e5763b8b9a63cda83006c7a.jpg,ant,ant,0.197799
../data/cleaned/ant/448b50c6778952be3df52a6a26affeff4844b7f4.jpg,ant,ant,0.202395
../data/cleaned/ant/451d80e25d8f18e8a68174ff8ff651c435bd4700.jpg,ant,ant,0.207965
../data/cleaned/ant/51782d78019c7b5be5333e7bc2dab3469f88fec7.jpg,ant,ant,0.200533
...,...,...,...
../data/cleaned/tick/c99e1bba324bc3b31bfab79473a50baeb0c87a2a.jpg,tick,ant,0.190451
../data/cleaned/tick/d842edcf4167941990cb0b05c8a3c58db001c8d6.jpg,tick,ant,0.203257
../data/cleaned/tick/ebeb916d8413d0b3387c84896fb99ddf6eb2fe13.jpg,tick,ant,0.204682
